# Aplicação Integrada de Monitoramento de Enchentes e Previsão de Surto de Malária - SentinelaMSF

Este notebook integra o fluxo de exploração/análise de dados e o pipeline de machine learning para previsão de surtos de malária, utilizando dados ambientais, históricos e de enchentes.  
**Futuras expansões** e melhorias estão detalhadas ao final do notebook.

---

## Importação de Bibliotecas e Carregamento dos Dados

Integra dados de malária, enchentes e clima já tratados no notebook de exploração.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.preprocessing import StandardScaler

import joblib
import os

## Carregamento e Integração dos Dados


In [ ]:
malaria_df = pd.read_csv('data/processed/malaria_tratada.csv')
enchentes_df = pd.read_csv('data/processed/enchentes_tratada.csv')
clima_df = pd.read_csv('data/processed/clima_tratado.csv')

df = malaria_df.merge(
    enchentes_df[['municipio', 'data', 'risco_enchente', 'dias_alagado']],
    on=['municipio', 'data'],
    how='left'
).merge(
    clima_df[['municipio', 'data', 'chuva_mm', 'temperatura_media', 'umidade_relativa']],
    on=['municipio', 'data'],
    how='left'
)

df['risco_enchente'] = df['risco_enchente'].fillna(0)
df['dias_alagado'] = df['dias_alagado'].fillna(0)
df['chuva_mm'] = df['chuva_mm'].fillna(df['chuva_mm'].median())
df['temperatura_media'] = df['temperatura_media'].fillna(df['temperatura_media'].median())
df['umidade_relativa'] = df['umidade_relativa'].fillna(df['umidade_relativa'].median())


## Definição do Alvo e Seleção de Features

Define como alvo a ocorrência de surto de malária (casos acima do percentil 75).

In [ ]:
limiar = df['casos'].quantile(0.75)
df['surto_malaria'] = (df['casos'] > limiar).astype(int)

features = [
    'risco_enchente',
    'chuva_mm',
    'temperatura_media',
    'umidade_relativa'
]
X = df[features]
y = df['surto_malaria']

---

## Divisão Treino/Teste e Pré-processamento

Normaliza as features numéricas e prepara para o treino do modelo.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


# Treinamento e Avaliação do Modelo

Treina um RandomForest e avalia o desempenho.

In [ ]:
models = {
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "MLP (Neural Net)": MLPClassifier(hidden_layer_sizes=(32,16), max_iter=500, random_state=42)
}

results = {}
for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    y_proba = model.predict_proba(X_test_scaled)[:, 1]
    results[name] = {
        "report": classification_report(y_test, y_pred, output_dict=True),
        "roc_auc": roc_auc_score(y_test, y_proba),
        "model": model
    }
    print(f"\n=== {name} ===")
    print(classification_report(y_test, y_pred))
    print("ROC AUC:", roc_auc_score(y_test, y_proba))

## Lógica Baseada em Regras (Baseline)

Regras simples baseadas em conhecimento:
- Se área ficou alagada por mais de 5 dias **e** risco de enchente presente **e** chuva acima da mediana, risco de surto é ALTO.
- Caso contrário, risco é BAIXO/MÉDIO.

Futuramente pretendo ajustar pelo indice de percentil pluviometrico.

In [ ]:
def regra_baseline(row):
    if (row['dias_alagado'] > 5) and (row['risco_enchente'] == 1) and (row['chuva_mm'] > df['chuva_mm'].median()):
        return 2  # ALTO
    elif (row['dias_alagado'] > 2) and (row['risco_enchente'] == 1):
        return 1  # MÉDIO
    else:
        return 0  # BAIXO

df['risco_regra'] = df.apply(regra_baseline, axis=1)
print(df['risco_regra'].value_counts())

## Importância das Variáveis (Random Forest)

In [ ]:
importances = pd.Series(results['Random Forest']['model'].feature_importances_, index=features)
importances.sort_values().plot(kind='barh')
plt.title('Importância das Variáveis')
plt.show()


## Salvando Modelos e Scaler

In [ ]:
os.makedirs('models', exist_ok=True)
for name, res in results.items():
    joblib.dump(res['model'], f'models/modelo_{name.replace(" ", "_").lower()}.pkl')
joblib.dump(scaler, 'models/scaler_surto_malaria.pkl')

## Interface Streamlit para Visualização e Alerta

Crie um arquivo `app.py` com o seguinte conteúdo para rodar o painel interativo:

**Para rodar:**
```sh
streamlit run app.py

# Expansões Futuras

- Automatizar processamento de imagens de satélite para detecção de áreas alagadas (NDWI, ML).
- Incluir variáveis ambientais adicionais e relações espaciais.
- Aprimorar lógica baseada em regras com conhecimento de especialistas.
- Integrar mapas interativos com localização das áreas prioritárias.


## Conclusão

Este notebook demonstrou a integração dos principais dados do projeto SentinelaMSF, permitindo análises espaciais e temporais para apoio à atuação do MSF na região Yanomami.
